In [8]:
import torch
import cv2
import os
from glob import glob
import numpy as np
import segmentation_models_pytorch as smp
from tqdm import tqdm

def preprocess_image(image):
    image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    lab[:, :, 0] = clahe.apply(lab[:, :, 0])
    image = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    kernel = np.array([[-1,-1,-1], [-1, 9,-1], [-1,-1,-1]])
    image = cv2.filter2D(image, -1, kernel)
    return image

print("="*80)
print("ROAD SEGMENTATION TEST SCRIPT")
print("="*80)

print("\n[1/5] Loading model...")
model = smp.Unet(
    encoder_name="mobilenet_v2",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None
)

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model = model.to(device)
model.eval()
print("✓ Model loaded successfully")

print("\n[2/5] Getting test images...")
test_dir = input("Enter test folder name: ").strip()

# Check if folder exists
if not os.path.exists(test_dir):
    print(f"ERROR: Folder '{test_dir}' does not exist!")
    print(f"Current directory: {os.getcwd()}")
    print(f"Available folders: {[d for d in os.listdir('.') if os.path.isdir(d)]}")
    exit()

# Try different extensions
test_images = sorted(glob(os.path.join(test_dir, '*.jpg')))
if len(test_images) == 0:
    test_images = sorted(glob(os.path.join(test_dir, '*.png')))
if len(test_images) == 0:
    test_images = sorted(glob(os.path.join(test_dir, '*.jpeg')))
if len(test_images) == 0:
    test_images = sorted(glob(os.path.join(test_dir, '*.JPG')))

if len(test_images) == 0:
    print(f"\nERROR: No images found in '{test_dir}'!")
    print(f"Files in folder: {os.listdir(test_dir)[:10]}")
    exit()

print(f"✓ Found {len(test_images)} test images")

output_dir = 'final_predicted_masks'
os.makedirs(output_dir, exist_ok=True)
print(f"✓ Output directory: {output_dir}/")

print("\n[3/5] Generating predictions...")
with torch.no_grad():
    for img_path in tqdm(test_images, desc='Processing'):
        image = cv2.imread(img_path)
        if image is None:
            print(f"Warning: Could not read {img_path}")
            continue
        
        original_size = image.shape[:2]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = preprocess_image(image)
        image_resized = cv2.resize(image, (512, 512))
        
        image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).unsqueeze(0)
        image_tensor = image_tensor.float() / 255.0
        image_tensor = image_tensor.to(device)
        
        output = model(image_tensor)
        pred_mask = torch.sigmoid(output).cpu().numpy()[0, 0]
        pred_mask = (pred_mask > 0.5).astype('uint8') * 255
        pred_mask = cv2.resize(pred_mask, (original_size[1], original_size[0]))
        
        filename = os.path.basename(img_path)
        output_filename = filename.replace('_sat.jpg', '.png').replace('.jpg', '.png').replace('.JPG', '.png')
        cv2.imwrite(os.path.join(output_dir, output_filename), pred_mask)

print("\n[4/5] Predictions complete!")
print(f"✓ {len(test_images)} masks saved in: {output_dir}/")
print("\n[5/5] Next steps:")
print("="*80)
print("   RESULT_DIR = 'predicted_masks'")
print("   GT_DIR = 'gt_masks'")
print("3. Run: python hackathon_checker.py")
print("="*80)


ROAD SEGMENTATION TEST SCRIPT

[1/5] Loading model...
✓ Model loaded successfully

[2/5] Getting test images...
✓ Found 300 test images
✓ Output directory: final_predicted_masks/

[3/5] Generating predictions...


Processing: 100%|██████████| 300/300 [02:02<00:00,  2.44it/s]


[4/5] Predictions complete!
✓ 300 masks saved in: final_predicted_masks/

[5/5] Next steps:
   RESULT_DIR = 'predicted_masks'
   GT_DIR = 'gt_masks'
3. Run: python hackathon_checker.py


In [10]:
import os
import sys
from typing import Tuple, List
import cv2
import numpy as np

#!/usr/bin/env python3
"""
hackathon_checker.py

Compare paired PNG segmentations in two folders (result vs ground truth).

- Edit RESULT_DIR and GT_DIR with hardcoded absolute or relative paths.
- Both folders must contain PNG files with identical names to be compared.
- Each image is:
    * read in grayscale
    * binarized with threshold 127 (>=128 -> white / 255)
    dice_scores = []
    * white foreground dilated with a circular 5x5 kernel
- For each pair the script computes:
    * IoU for foreground (white)
    * IoU for background (black)
    * mean IoU = (IoU_foreground + IoU_background) / 2
    * Dice score for foreground
- Prints per-image metrics and the overall average of mean IoU across the dataset.
"""


# === USER CONFIG: set hardcoded folders here ===
RESULT_DIR = "/Users/manishank/Desktop/cv-datathon/final_predicted_masks"      # <-- change to your results folder
GT_DIR     = "/Users/manishank/Desktop/cv-datathon/gt_300"  # <-- change to your ground-truth folder
# ================================================

KERNEL = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
THRESH = 127  # binarization threshold

def load_png_pairs(result_dir: str, gt_dir: str) -> List[Tuple[str, str, str]]:
    """Return list of (filename, result_path, gt_path) for matching PNG basenames."""
    res_files = {f for f in os.listdir(result_dir) if f.lower().endswith(".png")}
    gt_files = {f for f in os.listdir(gt_dir) if f.lower().endswith(".png")}
    common = sorted(res_files & gt_files)
    pairs = [(fname, os.path.join(result_dir, fname), os.path.join(gt_dir, fname)) for fname in common]
    return pairs

def read_binarize_dilate(path: str) -> np.ndarray:
    """Read image as grayscale, binarize at THRESH, dilate foreground, return boolean mask (True=foreground)."""
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Could not read image: {path}")
    # threshold: pixels > THRESH become 255; others 0
    _, th = cv2.threshold(img, THRESH, 255, cv2.THRESH_BINARY)
    # dilate foreground
    dil = cv2.dilate(th, KERNEL)
    mask = dil.astype(np.uint8) == 255
    return mask

def iou_and_dice(mask_a: np.ndarray, mask_b: np.ndarray) -> Tuple[float, float, float]:
    
    """
    Compute IoU_foreground, IoU_background, Dice_foreground between two boolean masks.
    Returns (iou_fg, iou_bg, dice_fg).
    """
    if mask_a.shape != mask_b.shape:
        raise ValueError("Masks must have the same shape for metric computation")
    a = mask_a.astype(np.bool_)
    b = mask_b.astype(np.bool_)
    # Foreground intersection / union
    inter_fg = np.logical_and(a, b).sum()
    union_fg = np.logical_or(a, b).sum()
    if union_fg == 0:
        iou_fg = 1.0  # both empty -> perfect overlap
    else:
        iou_fg = inter_fg / union_fg
    # Background: invert masks
    a_bg = np.logical_not(a)
    b_bg = np.logical_not(b)
    inter_bg = np.logical_and(a_bg, b_bg).sum()
    union_bg = np.logical_or(a_bg, b_bg).sum()
    if union_bg == 0:
        iou_bg = 1.0
    else:
        iou_bg = inter_bg / union_bg
    # Dice for foreground: 2*|A∩B| / (|A|+|B|)
    sum_sizes = a.sum() + b.sum()
    if sum_sizes == 0:
        dice_fg = 1.0
    else:
        dice_fg = 2 * inter_fg / sum_sizes
    return iou_fg, iou_bg, dice_fg

def main():
    pairs = load_png_pairs(RESULT_DIR, GT_DIR)
    if len(pairs) == 0:
        print("No matching PNG files found between the two folders.")
        sys.exit(1)

    mean_ious = []
    dice_scores = []
    print(f"Comparing {len(pairs)} images...\n")
    print("{:<40s} {:>8s} {:>8s} {:>10s}".format("filename", "IoU_fg", "IoU_bg", "Dice_fg"))
    print("-" * 70)
    for fname, res_path, gt_path in pairs:
        try:
            res_mask = read_binarize_dilate(res_path)
            gt_mask  = read_binarize_dilate(gt_path)
        except Exception as e:
            print(f"{fname}: ERROR reading or processing: {e}")
            continue
        if res_mask.shape != gt_mask.shape:
            print(f"{fname}: SKIP (size mismatch: {res_mask.shape} vs {gt_mask.shape})")
            continue
        iou_fg, iou_bg, dice_fg = iou_and_dice(res_mask, gt_mask)
        mean_iou = (iou_fg + iou_bg) / 2.0
        mean_ious.append(mean_iou)
        dice_scores.append(dice_fg)
        print("{:<40s} {:8.4f} {:8.4f} {:10.4f}".format(fname, iou_fg, iou_bg, dice_fg))

    if len(mean_ious) == 0:
        print("\nNo valid image pairs were processed.")
        sys.exit(1)

    overall_mean_iou = float(np.mean(mean_ious))
    print("\nOverall average of mean IoU across dataset: {:.6f}".format(overall_mean_iou))
    # Also print average Dice (foreground) across the dataset
    overall_mean_dice = float(np.mean(dice_scores)) if len(dice_scores) > 0 else 0.0
    print("Overall average Dice (foreground) across dataset: {:.6f}".format(overall_mean_dice))

if __name__ == "__main__":
    main()

Comparing 300 images...

filename                                   IoU_fg   IoU_bg    Dice_fg
----------------------------------------------------------------------
000109.png                                 0.7678   0.9957     0.8686
003635.png                                 0.2681   0.9825     0.4228
005747.png                                 0.6061   0.8832     0.7548
005928.png                                 0.3627   0.9199     0.5323
005933.png                                 0.5447   0.9448     0.7052
007830.png                                 0.5013   0.9860     0.6678
010339.png                                 0.1921   0.9695     0.3223
010638.png                                 0.5555   0.9573     0.7142
025567.png                                 0.7121   0.9770     0.8318
025983.png                                 0.3431   0.9843     0.5109
029017.png                                 0.5405   0.9894     0.7017
029135.png                                 0.6287   0.9867     0